In [ ]:
%reset
###--- import packages
import nilearn, scipy, os, bct
from nilearn import input_data, signal, plotting
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

In [ ]:
###--- Enter BIDS directory.
MainDir = '/home/connectomics/LearningBrain'

###--- Import task-data (resting state and dual n-back) ".nii" files and confounds - save list of locations.
rest_filenames = []
dual_filenames = []
rest_confounds = []
dual_confounds = []
for dirpath, dirnames, filenames in os.walk(MainDir):
    for filename in filenames:
        if ('rest_bold_space-MNI152NLin2009cAsym_variant-smoothAROMAnonaggr_preproc.nii.gz' in filename) and (MainDir)+'/derivatives/fmriprep/sub' in os.path.join(dirpath, filename):
            rest_filenames.append(os.path.join(dirpath, filename))
        if ('dualnback_bold_space-MNI152NLin2009cAsym_variant-smoothAROMAnonaggr_preproc.nii.gz' in filename) and (MainDir)+'/derivatives/fmriprep/sub'  in os.path.join(dirpath, filename):
            dual_filenames.append(os.path.join(dirpath, filename))
        if ('rest_bold_confounds.tsv' in filename) and (MainDir)+'/derivatives/fmriprep/sub' in os.path.join(dirpath, filename):
            rest_confounds.append(os.path.join(dirpath, filename))
        if ('dualnback_bold_confounds.tsv' in filename) and (MainDir)+'/derivatives/fmriprep/sub' in os.path.join(dirpath, filename):
            dual_confounds.append(os.path.join(dirpath, filename))
dual_confounds.sort()
dual_filenames.sort()
rest_confounds.sort()
rest_filenames.sort()
print('number of rest connectivity files is equal ' +str(len(rest_filenames)))
print('number of rest confound files is equal ' +str(len(rest_confounds)))
print('number of dual connectivity files is equal ' +str(len(dual_filenames)))
print('number of dual confound files is equal ' +str(len(dual_confounds)))


In [ ]:
#######--- HYPERPARAMETERS ---#######
###--- Connectivity measures
models = ['correlation', 'partial correlation', 'covariance', 'precision']

###--- Parcellations
power = nilearn.datasets.fetch_coords_power_2011()
dosenbach = nilearn.datasets.fetch_coords_dosenbach_2010()
multiscale = nilearn.datasets.fetch_atlas_basc_multiscale_2015()
aal = nilearn.datasets.fetch_atlas_aal()
harvard = nilearn.datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr0-1mm',symmetric_split=True)
#--- Create atlases as spheres from Power and Dosenbach coordinates
coords_power = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T
pow_masker = nilearn.input_data.NiftiSpheresMasker(seeds=coords_power, radius=3, standardize=True, detrend=True)
coords_dosenbach = np.vstack((dosenbach.rois['x'], dosenbach.rois['y'], dosenbach.rois['z'])).T
dos_masker = nilearn.input_data.NiftiSpheresMasker(seeds=coords_dosenbach, radius=3, standardize=True, detrend=True)
#--- Load Harvard, AAL, Multiscale atlases as masks
har_masker = nilearn.input_data.NiftiLabelsMasker(labels_img = harvard.maps, standardize=True, detrend=True)
aal_masker = nilearn.input_data.NiftiLabelsMasker(labels_img = aal.maps, standardize=True, detrend=True)
mul_masker = nilearn.input_data.NiftiLabelsMasker(labels_img = multiscale.scale197, standardize=True, detrend=True)
#--- Final list of maskers to iterate over.
maskers = [pow_masker, dos_masker, mul_masker, aal_masker, har_masker]
masker_names=['pow','dos','mul','aal','har']
print('Atlas preparation complete.')

In [ ]:
###--- Choose the data to work on (only resting-state)
func_filenames = rest_filenames
confounds = rest_confounds

###--- Denoising
#--- choose only the important regressors (CSF, White Matter, Linear Trend) and save new confounds file
for file,confound in zip(func_filenames,confounds):
    conf=pd.read_csv(confound,delimiter='\t')
    conf=conf[conf.filter(regex='WhiteMatter|CSF|CompCor|X|Y|Z').columns]
    np.savetxt(confound[:-4]+'_edited.csv',conf,delimiter=',')
print('Choice of important regressors complete.')

In [ ]:
###--- Iterate over all hyperparameters and generate a functional connectivity matrix for each set of parameters.
for file,confound in zip(func_filenames,confounds):
    for model in models: #models:
        for masker, masker_name in zip(maskers, masker_names):
            #--- Create time-series in all ROIs and apply filters
            time_series_power = masker.fit_transform(file, confounds=confound[:-4]+'_edited.csv')
            time_series_power_filtered=signal.clean(time_series_power,low_pass=0.08,high_pass=0.009)
            #--- Create the connectivity matrices
            correlation_measure = ConnectivityMeasure(kind = model)
            correlation_matrix_raw = correlation_measure.fit_transform([time_series_power_filtered])[0]
            #--- Mask the main diagonal:
            np.fill_diagonal(correlation_matrix_raw, 0)
            #--- Transform matrix into vector and apply z-score
            indices = np.triu_indices_from(correlation_matrix_raw,k=1)
            correlation_vec = np.asarray(correlation_matrix_raw[indices])
            zscore = scipy.stats.zscore(correlation_vec)
            correlation_matrix = np.zeros((len(correlation_matrix_raw),len(correlation_matrix_raw)))      
            correlation_matrix[indices]=correlation_vec
            correlation_matrix = correlation_matrix + correlation_matrix.T
            ###--- SAVE RESULTS 
            #--- extract sub number from file name 
            sub = int(file[file.find('sub-')+4:file.find('sub-')+6])
            #--- extract session number from file name
            sess = int(file[file.find('ses-')+4:file.find('ses-')+5])
            # plotting matrices to files
            display = plotting.plot_matrix(correlation_matrix, vmin=-0.8, vmax=0.8, colorbar=True)    
            display.figure.savefig('matKFLB_sub'+'{:02d}'.format(sub)+ '_ses' + str(sess) + '_' + masker_name[:3] + '_' + model[:3] + '.png')     
            plt.clf()
            # plotting histograms of data
            plt.hist(np.ravel(correlation_matrix), bins=100)
            plt.savefig('matKFLB_sub'+'{:02d}'.format(sub)+ '_ses' + str(sess) + '_' + masker_name[:3] + '_' + model[:3] + 'histogram.png')
            plt.clf()
            #--- save connectivity matrix as human-readable data (TXT)
            np.savetxt('matKFLB_sub'+'{:02d}'.format(sub)+ '_ses' + str(sess) + '_' + masker_name[:3] + '_' + model[:3] + '.txt', correlation_matrix)
            print('Subject ' + str(sub) + ', session ' + str(sess) + ', model: '+ str(model) +' and atlas: '+str(masker_name)+' complete.')